In [ ]:
import numpy as np
import pandas as pd
import squidpy as sq
import scipy.sparse as sp

# Dataset test

In [ ]:
def read_dataset(input_exp, input_adj, filter_num=None, add_number=None):
    exp = pd.read_csv(open(input_exp))
    adj = pd.read_csv(open(input_adj))

    if not (adj.values==adj.values.T).all():
        raise ImportError('The input adjacency matrix is not a symmetric matrix, please check the input.')
    if not np.diag(adj.values).sum()==0:
        raise ImportError('The diagonal elements of input adjacency matrix are not all 0, please check the input.')

    if filter_num is not None:
        exp = filter_gene(exp, filter_num)
    if add_number is not None:
        exp = log_transform(exp, add_number)

    return exp, adj

In [ ]:
exp_df, adj_df = read_dataset("./dataset/seqFISH/counts.csv", "./dataset/seqFISH/adj.csv", None, None)
exp, adj = exp_df.values, adj_df.values

In [ ]:
exp

array([[ 1.83,  1.52,  2.41, ..., -1.4 ,  2.51,  0.6 ],
       [ 0.38, -0.04,  0.59, ...,  0.22,  1.88,  2.05],
       [ 0.82,  0.12,  0.67, ...,  0.15,  3.2 ,  2.3 ],
       ...,
       [-1.06, -0.18, -0.33, ..., -0.49,  0.8 ,  0.39],
       [-1.99, -0.25,  1.34, ..., -0.14, -2.66,  0.37],
       [-1.69,  0.94, -0.12, ...,  0.04, -0.61, -0.14]])

In [ ]:
adj

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
adata = sq.datasets.visium_fluo_adata()
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
adata.obsp["spatial_connectivities"].A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import scipy.sparse as sp
import numpy as np

In [ ]:
import squidpy as sq
import torch
adata = sq.datasets.visium_fluo_adata()
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=200)
adj_mtx = adata.obsp["spatial_connectivities"]
type(adata.X)
#features = torch.tensor(adata.X.A)
#n_nodes = adj_mtx.shape[0]
#n_input = features.size(1)

scipy.sparse._csr.csr_matrix

In [ ]:
sparse_mx = adata.X[:5, :100].tocoo()

In [ ]:
sparse_mx.shape

(5, 100)

In [ ]:
values = sparse_mx.data
indices = np.vstack((sparse_mx.row, sparse_mx.col))

In [ ]:
a= torch.FloatTensor(values)

In [ ]:
b= torch.IntTensor(indices)

In [ ]:
a.size()

torch.Size([118])

In [ ]:
b.size()

torch.Size([2, 118])

In [ ]:
shape = sparse_mx.shape

In [ ]:
test = torch.sparse_coo_tensor(b, a, torch.Size(shape))

In [ ]:
def sparse_matrix_to_sparse_tensor(sparse_mx):

    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()

    values = sparse_mx.data
    indices = np.vstack((sparse_mx.row, sparse_mx.col))

    i = torch.IntTensor(indices)
    v = torch.FloatTensor(values)
    shape = sparse_mx.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [ ]:
test_tensor = sparse_matrix_to_sparse_tensor(sparse_mx)

In [ ]:
sparse_test_tensor = torch.sparse_coo_tensor(adata.X)

TypeError: sparse_coo_tensor(): argument 'size' (position 1) must be tuple of ints, not csr_matrix

In [ ]:
type(adata.X)

scipy.sparse._csr.csr_matrix

In [ ]:
type(sparse_test_tensor)

torch.Tensor

In [ ]:
adata.obsp["spatial_connectivities"].A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [ ]:
type(adata.obsp["spatial_connectivities"].todense())

numpy.matrix

In [ ]:
import numpy as np
np.diag(adata.obsp["spatial_connectivities"].todense()).sum() == 0

True

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0178,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.1245,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.1538, 0.0000, 0.0000]])

In [ ]:
adata.X.A.shape

(2800, 16562)

In [ ]:
import squidpy as sq
import torch
adata = sq.datasets.visium_fluo_adata()
len(adata.X.A)

2800

In [ ]:
features.size()

In [ ]:
type(adata.obsp["spatial_connectivities"])

## Functionality Test

In [ ]:
import anndata as ad
import numpy as np
import scipy.sparse as sp

from deeplinc.data import train_test_split
from deeplinc.data import sparse_adj_mx_to_edges
from deeplinc.data import sample_neg_test_edges
from deeplinc.data import has_overlapping_edges
from deeplinc.data import normalize_adj_mx

In [ ]:
adj_mx_test = sp.csr_matrix(np.array([[0, 1, 1, 0, 0],
                                      [1, 0, 0, 0, 0],
                                      [1, 0, 0, 0, 0],
                                      [0, 0, 0, 0, 1],
                                      [0, 0, 0, 1, 0]]))

In [ ]:
sparse_adj_mx_to_edges(adj_mx_test)

array([[0, 1],
       [0, 2],
       [1, 0],
       [2, 0],
       [3, 4],
       [4, 3]], dtype=int32)

In [ ]:
adj_mx_key = "spatial_connectivities"
test_ratio = 0.1

## Extract edges from the adjacency matrix
adj_mx = adata.obsp[adj_mx_key]
n_nodes = adj_mx.shape[0]
adj_mx_triu = sp.triu(adj_mx)  # upper triangle of adjacency matrix
# single edge for adjacent cells
edges_single = sparse_adj_mx_to_edges(adj_mx_triu)
# double edge for adjacent cells
edges_double = sparse_adj_mx_to_edges(adj_mx)
n_edges = edges_single.shape[0]
if test_ratio > 1: # absolute test ratio
    test_ratio = test_ratio / n_edges 
## Split into train and test edges
n_edges_test = int(np.floor(n_edges * test_ratio))
idx_edges_all = np.array(range(n_edges))
np.random.shuffle(idx_edges_all)
idx_edges_test = idx_edges_all[:n_edges_test]
edges_test = edges_single[idx_edges_test]
edges_train = np.delete(edges_single, idx_edges_test, axis=0)
## Sample negative test edges
# node combinations without edge
n_nonedges = n_nodes**2-int(adj_mx.sum())-n_nodes
if (n_nonedges)/2 < 2*n_edges_test:
    raise AssertionError("The network is too dense. Please decrease the \
    test ratio or delete some edges in the network.")
else:
    edges_test_neg = sample_neg_test_edges(n_nodes,
                                           edges_test,
                                           edges_double)
## Construct (symmetric) train and test adjacency matrix with sampled edges
adj_train = sp.csr_matrix(
    (np.ones(edges_train.shape[0]), (edges_train[:, 0], edges_train[:, 1])),
    shape=adj_mx.shape,
)
adj_train = adj_train + adj_train.T # make symmetric
adj_test = sp.csr_matrix(
    (np.ones(edges_test.shape[0]), (edges_test[:, 0], edges_test[:, 1])),
    shape=adj_mx.shape,
)
adj_test = adj_test + adj_test.T # make symmetric

KeyboardInterrupt: 

In [ ]:
has_overlapping_edges(edges_test[1], edges_test)

In [ ]:
edges_test_neg = sample_neg_test_edges(n_nodes, edges_test, edges_double)

In [ ]:
edges_test_neg

In [ ]:
len(edges_test)

In [ ]:
len(edges_test_neg)

In [ ]:
has_overlapping_edges(edges_test_neg, edges_test)

In [ ]:
adj_train, adj_test, edges_train, edges_test, edges_test_neg  = train_test_split(adata, test_ratio=0.1)

In [ ]:
adj_train

In [ ]:
adj_test

In [ ]:
edges_train.shape[0] + edges_test.shape[0]

In [ ]:
edges_train.shape[0]

In [ ]:
edges_test.shape[0]

In [ ]:
edges_test_neg

In [ ]:
has_overlapping_edges(edges_test_neg, edges_train)

In [ ]:
split_adj_mx_and_edges = train_test_split(adata, test_ratio=0.1)

In [ ]:
adj_mx_train, adj_mx_test = split_adj_mx_and_edges[:2]
edges_train, edges_test, edges_test_neg = split_adj_mx_and_edges[2:]

In [ ]:
normalize_adj_mx(adj_mx_train)

In [ ]:
adj_mx_train.shape[0]

In [ ]:
adj_mx_train.A

In [ ]:
sp.eye(adj_mx_train.shape[0])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create data
N = 50
g1 = (0.2*np.random.rand(N), 0.25*np.random.rand(N))
g2 = (0.5+0.1 * np.random.rand(N), 0.4+0.2*np.random.rand(N))
g3 = (0.5+0.1 * np.random.rand(N), 0.4*np.random.rand(N))
g4 = (0.4+0.1 * np.random.rand(N), 0.3*np.random.rand(N))
g5 = (0.6+0.1 * np.random.rand(N), 0.5*np.random.rand(N))

data = (g1, g2, g3, g4, g5)
colors = ("green", "purple", "orange", "olive", "red")
groups = ("healthy", "latent_tb", "treatment_1m", "treatment_2m", "active_tb")

# Create plot
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for data, color, group in zip(data, colors, groups):
    x, y = data
    ax.scatter(x, y, alpha=0.8, c=color, edgecolors='none', s=30, label=group)

plt.title('UMAP: TB Disease Trajectory')
plt.legend(loc=2)
plt.show()

In [2]:
import scanpy as sc
import squidpy as sq

import numpy as np

In [3]:
adata = sq.datasets.visium_fluo_adata()
adata

  0%|          | 0.00/242M [00:00<?, ?B/s]

AnnData object with n_obs × n_vars = 2800 × 16562
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)

In [7]:
adata.obsp["spatial_connectivities"].A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
adata

AnnData object with n_obs × n_vars = 2800 × 16562
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial', 'umap', 'spatial_neighbors'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [32]:
import numpy as np
import torch
from scipy import sparse
from torch.utils.data import Dataset

class SpatialAnnDataDataset(Dataset):
    """

       Parameters
       ----------
       adata: `~anndata.AnnData`
            Spatially annotated AnnData object. Adjaceny matrix needs to be stored in adata.obsp["spatial_connectivities"].
    """

    def __init__(self,
                 adata,
                 ):

        if sparse.issparse(adata.X):
            self.X = torch.tensor(adata.X.A)
        else:
            self.X = torch.tensor(adata.X)

        if sparse.issparse(adata.obsp["spatial_connectivities"]):
            self.A = torch.tensor(adata.obsp["spatial_connectivities"].A)
        else:
            self.A = torch.tensor(adata.obsp["spatial_connectivities"]) # output from squidpy.gr.spatial_neighbors

        if not (self.A==self.A.T).all():
            raise ImportError("The input adjacency matrix is not a symmetric matrix.")
        if not np.diag(self.A).sum()==0:
            raise ImportError("The diagonal elements of input adjacency matrix are not all 0.")


    def __getitem__(self, index):
        output = dict()

        output["X"] = self.X[index, :]
        output["A"] = self.A[index, :]

        return output


    def __len__(self):
        return self.X.size(0)


    def __str__(self):
        return self.__name

True

In [20]:
sparse.issparse(adata.obsp["spatial_connectivities"])

True

In [9]:
import torch
from scipy import sparse
from torch.utils.data import Dataset

In [35]:
import numpy as np
import torch
from scipy import sparse
from torch.utils.data import Dataset

class SpatialAnnDataDataset(Dataset):
    """

       Parameters
       ----------
       adata: `~anndata.AnnData`
            Spatially annotated AnnData object. Adjaceny matrix needs to be stored in adata.obsp["spatial_connectivities"].
    """

    def __init__(self,
                 adata,
                 ):

        if sparse.issparse(adata.X):
            self.X = torch.tensor(adata.X.A)
        else:
            self.X = torch.tensor(adata.X)

        if sparse.issparse(adata.obsp["spatial_connectivities"]):
            self.A = torch.tensor(adata.obsp["spatial_connectivities"].A)
        else:
            self.A = torch.tensor(adata.obsp["spatial_connectivities"]) # output from squidpy.gr.spatial_neighbors

        if not (self.A==self.A.T).all():
            raise ImportError("The input adjacency matrix is not a symmetric matrix.")
        if not np.diag(self.A).sum()==0:
            raise ImportError("The diagonal elements of input adjacency matrix are not all 0.")


    def __getitem__(self, index):
        output = dict()

        output["X"] = self.X[index, :]
        output["A"] = self.A[index, :]

        return output


    def __len__(self):
        return self.X.size(0)


    def __str__(self):
        return self.__name

In [51]:
dataset = SpatialAnnDataDataset(adata)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [2000, 800])

In [52]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [53]:
type(train_dataset)

torch.utils.data.dataset.Subset

In [55]:
next(iter(train_dataloader))

{'X': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.4659],
         [0.0000, 0.0000, 0.7449,  ..., 0.0000, 0.0000, 1.1671],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9113],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.1020],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2614]]),
 'A': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)}

In [40]:
test_dataset[5:12]

{'X': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.5867],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.5107],
         [0.7439, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.8750],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2222],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.7781],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'A': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)}

In [39]:
len(test_dataset)

2800